<a href="https://colab.research.google.com/github/Sahanasd2003/Telecom-churn-prediction/blob/main/Telecom_churn_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
blastchar_telco_customer_churn_path = kagglehub.dataset_download('blastchar/telco-customer-churn')

print('Data source import complete.')


<h2><center> Telecom customer churn analysis </center></h2>

<h3 style="background-color: #f78239 ;padding: 15px ; margin: 7px;" align="justify">Overview.</h3>

 <p> In this notebook we will analyze the telecom customer churn dataset, firstly we will conduct data exploratory analysis and univarient / bivarient analysis , afetr that we will creat a model in order to predict wether a specific customer will churn , finally will provide inights and conclusion.</p>


<h3 style="background-color: #f78239 ;padding: 15px ; margin: 7px;" align="justify"> About the data.</h3>

the dataset we will use is a public dataset which can be found and downloaded from[ Here](https://www.kaggle.com/datasets/blastchar/telco-customer-churn).
    
  Each row represents a customer, each column contains customer’s attributes described on the column Metadata.

The data set includes information about:
* Customers who left within the last month – the column is called Churn
* Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
* Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
* Demographic info about customers – gender, age range, and if they have partners and dependents
    

<h3 style="background-color: #f78239 ;padding: 15px ; margin: 7px;" align="justify"> Notebook set up.</h3>
first  thing to do is to import the set of libraries we are going to use in order to analyse and model our data.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import plotly
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split ,KFold,StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report ,confusion_matrix,ConfusionMatrixDisplay
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import roc_curve, auc ,roc_auc_score
import optuna

#### Load data

In [ ]:
data = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

#### Data wrangling

In [ ]:
data.head()

In [ ]:
data.info()
print("data shape is :",data.shape)

 As we can see  our data consist of 7043 rows and 21 columns which makes it quite small dataset. The variables are in an object type exept for SeniorCitizen  , tenure are both int64 and MonthlyCharges has a float64 type.

In [ ]:
# Check for missing data and duplicated customer_IDs
data.isna().any().any(), data.customerID.duplicated().any()

There are no dublicated and no missing values in our data, which means we can start analyze it direcyly

<h3 style="background-color: #f78239 ;padding: 15px ; margin: 7px;" align="justify"> Explotory data analysis.</h3>
we will take a look at each variable and check it's distrubution and how it relates with the target.


**Basic summary statistics for numirical variables**

In [ ]:
data.describe().style.background_gradient()

#### Churn by Gender

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(data=data, x="gender", hue="Churn", multiple="dodge", palette ='flare',shrink=.7)
plt.title('Churn by Gender')

The gender variable is ballanced in our dataset (there are exactly 3555 male and 3488 female ) , looking at the figure above we can see that churn is equally present with in female as in males.

#### Churn by senior Citizen

Senior Citizen a binary feature that indicates wether the customer is an an elderly person or not. looking at the figure below we can see that the dataset contains less senior citizens ( 1142 from the total 7043 custormers). we can also see that churn is present with in both categories.

In [ ]:
plt.figure(figsize=(9,5))
sns.histplot(x= "SeniorCitizen", data=data,shrink=7 ,  palette ='BuGn_r',hue = "SeniorCitizen");
plt.figure(figsize=(11,5))
sns.histplot(data=data, x="SeniorCitizen", hue="Churn", multiple="dodge", palette ='BuGn_r',shrink=7)
plt.title('Churn by gender')

#### Partnership variable
the partner feature is a binary feature that indicates whether the customer has a partner or not. the figures bellow show how this variable is present in our data and how it realates to churn.

In [ ]:
plt.figure(figsize=(9,5))
sns.histplot(x= "Partner", data=data,shrink=.7 ,  palette ='YlOrBr',hue = "Partner");
plt.figure(figsize=(11,5))
sns.histplot(data=data, x="Partner", hue="Churn", multiple="dodge", palette ='YlOrBr',shrink=.7)
plt.title('Churn by gender')

#### Dependents varibale
the Dependents feature is a binarry variable that tells wether the cutomer is a depandent or not. and we can see that most customers aren't dependent while churn is fairly present in both types.

In [ ]:
plt.figure(figsize=(9,5))
sns.histplot(data=data,x="Dependents",hue='Churn', palette='rocket',shrink=.5)
plt.title('Dependents feature')

#### Tenure Feature
looking at the figure below we can tell that the less tenure the customer has the more likely he is about to churn.


In [ ]:
plt.figure(figsize=(12,8))
sns.histplot(data=data,x="tenure",hue='Churn', palette='BuPu')
plt.title('Tenure feature')

####  Telecom services
Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies.The figures below shows the distribution of each service and how it affects shurn ( we can't make conclusion here since we are just exploring the dataset).

In [ ]:
fig, ax = plt.subplots(2, 2,figsize=(15,12))
sns.histplot(data=data,x="PhoneService",hue='Churn', palette='hsv',ax=ax[0,0])
sns.histplot(data=data,x="MultipleLines",hue='Churn', palette='magma',ax=ax[0,1])
sns.histplot(data=data,x="InternetService",hue='Churn', palette='mako',ax=ax[1,0])
sns.histplot(data=data,x='OnlineSecurity',hue='Churn', palette='prism',ax=ax[1,1])


In [ ]:
fig, ax = plt.subplots(2,2,figsize=(15,12))
sns.histplot(data=data,x="OnlineBackup",hue='Churn', palette='hsv',ax=ax[0,0])
sns.histplot(data=data,x="DeviceProtection",hue='Churn', palette='magma',ax=ax[0,1])
sns.histplot(data=data,x="TechSupport",hue='Churn', palette='prism',ax=ax[1,0])
sns.histplot(data=data,x='StreamingTV',hue='Churn', palette='mako',ax=ax[1,1])


### Contract Feature
the contract variable tells how long the individual have been a customer , there are three tpype of contract ; Month-to-month (3875 customers) , Two years (1695) and one year (1473) , we can easily see that those who has month-to-month contract are more likely to churn.


In [ ]:
plt.figure(figsize=(9,5))
sns.histplot(data=data,x="Contract",hue='Churn', shrink =.7 , palette='copper')
plt.title('Contract feature')

#### PaperlessBilling
paperless billing is also a binary feature which tell whether the customer has activated the paperless billing method or not.

In [ ]:
plt.figure(figsize=(9,5))
sns.histplot(data=data,x="PaperlessBilling",hue='Churn', palette='Set2',shrink=.5)
plt.title('PaperlessBilling feature')

### PaymentMethod
PaymentMethod feature tells what's the method that customer use to pay his bills , there are four methodes of payment ; Electronic check , mailed check , bank transfer and credit card.

In [ ]:
plt.figure(figsize=(11,5))
sns.histplot(data=data,x="PaymentMethod",hue='Churn', palette='tab20c_r',shrink =.6)
plt.title('PaymentMethod feature')

### MonthlyCharges

In [ ]:
plt.figure(figsize=(9,5))
sns.histplot(data=data,x="MonthlyCharges", palette='hsv_r',hue ="Churn")

#### Target Distribution

In [ ]:
plt.figure(figsize=(9,5))
sns.countplot(data=data,x="Churn", palette='gist_heat',)
plt.title('Target Distribution ')

<h3 style="background-color: #f78239 ;padding: 15px ; margin: 7px;" align="justify"> Churn prediction.</h3>

Now after we did some analysis on our data , we are going to creat a model it using Xgboost , based on features that enables us to predict weather the custumer is likely to churn in the next month.

### Data preparation

In [ ]:
X = data.drop(columns=['customerID',"Churn"]).copy() #drop ID and target variable
y = data["Churn"].copy()

In [ ]:
cat_col = [col for col in X.columns if X[col].dtype == 'object']
num_col = [col for col in X.columns if X[col].dtype == 'int']

In [ ]:
## label encoder for categorical features
label_encoder = LabelEncoder()
for col in cat_col:
        X[col] = label_encoder.fit_transform(X[col])

y_encoded= label_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train,y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
print(X_train.shape,X_test.shape, y_train.shape,y_test.shape )

#### Training and prediction
Now we have the train and test data all set we go on to devlop our model.

In [ ]:
%%time
splits = 5 # number of folds
predictions = [] # list of predictions
scores = [] # auc_roc validation score
feat_imp = pd.DataFrame() # get feat importance
train_x = X_train.values ## convert Df to numpy array
kf =  StratifiedKFold(n_splits=splits, shuffle=True, random_state=42) # creat folds
###training extra trees classifier ###
for fold, (idx_train, idx_valid) in enumerate(kf.split(train_x,y_train)):
    X_tr, y_tr = train_x[idx_train], y_train[idx_train]
    X_val, y_val = train_x[idx_valid], y_train[idx_valid]
    model =  XGBClassifier(random_state = 42)
    model.fit(X_tr,y_tr)
    val_pred = model.predict_proba(X_val)[:, 1]
    score = roc_auc_score(y_val, val_pred)
    scores.append(score)
    print(f"Fold: {fold + 1} roc_auc Score is : {score}")
    fold_imp= pd.DataFrame()
    fold_imp["Feature"] = X_test.columns
    fold_imp["importance"] = model.feature_importances_
    fold_imp["fold"] = fold+ 1
    feat_imp = pd.concat([feat_imp, fold_imp], axis=0)
    print('*'*40)
    test_preds = model.predict_proba(X_test)[:, 1]
    predictions.append(test_preds)
print(f" mean Validation roc_aucis : {np.mean(scores)}")

### Feature importance
Feature Importance is a list of our features and their score. It represent represent the importance of each variable. A higher score means that the specific feature will have a larger effect on the model.

In [ ]:
plot =feat_imp.groupby("Feature").mean().reset_index()
plt.figure(figsize=(18,10))
sns.barplot(x="importance", y="Feature", data=plot.sort_values(by="importance", ascending=False),palette = 'GnBu_r')
plt.title('XGB Features importance ')
plt.tight_layout()
plt.show()

#### Test Roc_auc score

In [ ]:
preds = np.array(predictions)
preds = preds.mean(axis=0)
test_score = roc_auc_score(y_test, preds)
test_score

#### Classification_report
[classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) from scikit let's us quickly build a text report showing the main classification metrics.

In [ ]:
preds=np.around(np.around(preds, decimals=0)) ## round values to 0 and 1
print(classification_report(y_test,preds))

#### Confusion_matrix
[A confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) is a table that is used to define the performance of a classification algorithm.

In [ ]:
cm_graph = confusion_matrix(y_test, preds)
cm_display = ConfusionMatrixDisplay(cm_graph).plot()

Using the deafault Xgboost paramaters we got validation score of 0.81 and a test score of 0.84 which is an excellent discrimination , although we can finde tune the model and improve the score , we can be confident about our model predicting on new data.

### Summary  
In this notebook we analyzed the telecom dataset , we first did some data exploration and then we conducted Univariate analysis.

Our goal was to check for trends and insghits each variable can provid, after that we traind an Xgboost classifier using 5 folds which appeared to be a good model for our dataset.

Based on the analysis and feature importance of our model we could say that There is a trend that customers who are on a month to month contract are more likely to churn, one way to reduce the churn rate is to advrtize for the other type of contracts or slowely  break the Month-to-Month contract.